In [10]:
from bs4 import BeautifulSoup
import requests
import urllib
import urllib.request

In [11]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'



r = requests.get(url)
soup = BeautifulSoup(r.content,'lxml')
#print(soup.prettify())

In [12]:
table = soup.find('table', class_ = 'wikitable sortable')
#print(match.prettify())

header = table.tbody.tr.th.text
print(header)

Postcode


In [13]:
for table in soup.find_all('table', class_ = 'wikitable sortable'):
    header = table.tbody.tr.text
    print(header)
header = header.rstrip('\n')
header = header.lstrip('\n')
header = header.replace('\n', ',')
header_list = header.split(',')
header_list


Postcode
Borough
Neighbourhood



['Postcode', 'Borough', 'Neighbourhood']

In [15]:
entry_list = []
count = 0
for table in soup.find_all('table', class_ = 'wikitable sortable'):
    entry = table.tbody.text
    entry = entry.rstrip('\n')
    entry = entry.rstrip(' ')
    entry = entry.lstrip('\n')
    entry = entry.lstrip(' ')
    entry = entry.replace('\n', ',')
    
    entry_list.append(entry)
    entry_list = entry.split(',')
    print()

In [16]:
#clean it up a bit
for element in entry_list:
    try:
        entry_list.remove('')
    except ValueError:
        pass
#entry_list
del entry_list[0:3] # we already have these in header_list
entry_list[0:10]

['M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A',
 'North York',
 'Parkwoods',
 'M4A']

In [17]:
import pandas as pd
import numpy as np

In [34]:
df_toronto = pd.DataFrame(data = np.array(entry_list).reshape(287,3), columns = header_list)
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [35]:

df_toronto.replace('Not assigned', np.nan, inplace=True)
df_toronto = df_toronto.dropna(subset=['Borough'])
print(df_toronto.shape)
df_toronto.head(10)

(210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,NaN
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [36]:

df_toronto.fillna(0, inplace=True)
df_toronto.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,0
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [37]:

mask = df_toronto['Neighbourhood'] == 0
df_toronto.loc[mask, 'Neighbourhood'] = df_toronto['Borough']
df_toronto['Neighbourhood'] = df_toronto['Neighbourhood'].mask(mask, df_toronto['Borough'])
print(df_toronto.shape)
df_toronto.head(10)

(210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [38]:

df_toronto_grouped = df_toronto.groupby(['Postcode', 'Borough']).agg({'Neighbourhood':lambda x: ', '.join(x)}).reset_index()
df_toronto_grouped.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [40]:

df_toronto_grouped

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [41]:

df_toronto_grouped.shape

(103, 3)

In [44]:
url2 = 'http://cocl.us/Geospatial_data'
df_ll = pd.read_csv(url2)
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [45]:
df_ll.rename({'Postal Code':'Postcode'}, axis=1, inplace=True)
df_ll.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [46]:
df_toronto_ll = pd.merge(df_toronto_grouped, df_ll, on='Postcode')
df_toronto_ll.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
